In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import random
import sympy
import scipy.stats as st
import scipy.special
import pandas
import csv
import math
import multiprocessing as mp
import datetime
from decimal import *
from mpl_toolkits.mplot3d import Axes3D
from google.colab import drive

## **Google Drive**

In [2]:
# connect to google drive to save the csv file
drive.mount('drive')
os.chdir("drive/My Drive/Yen-Ting Lin/Code")

Mounted at drive


## **Back-off Calculation**

### **Retransmission at same RAP**

In [3]:
# alpha with i
def find_kmin(T_RAREP, T_RAR, W_RAR, W_BO, i):
  return np.ceil((i - 1) + ((1 - (T_RAR + W_RAR + W_BO)) / T_RAREP))
  # return np.ceil((i - 1) - (((T_RAR + W_RAR + W_BO) - 1) / T_RAREP)) # same equation

def find_kmax(T_RAREP, T_RAR, W_RAR, W_BO, i):
  return np.floor(i - ((T_RAR + W_RAR + 1) / T_RAREP))

def find_alpha(T_RAREP, T_RAR, W_RAR, W_BO, i, k, kmin, kmax):
  if kmin <= k <= (i - ((T_RAR + W_RAR + W_BO) / T_RAREP)):
    return (((k - 1) * T_RAREP) + T_RAR + W_RAR + W_BO - ((i - 2) * T_RAREP)) / W_BO
  elif (i - ((T_RAR + W_RAR + W_BO) / T_RAREP)) < k < ((i - 1) - ((T_RAR + W_RAR) / T_RAREP)):
    return ((T_RAREP) / W_BO)
  elif ((i - 1) - ((T_RAR + W_RAR) / T_RAREP)) <= k <= kmax:
    return (((i - 1) * T_RAREP) - (((k - 1) * T_RAREP) + T_RAR + W_RAR)) / W_BO
  else:
    return 0

### **Retransmission of fallback RAP**

In [4]:
def find_jmin(T_RAREP_4sra, T_RAREP_2sra, T_RAR, W_RAR, i):
  return np.ceil((((i - 2) * T_RAREP_4sra) + T_RAR + W_RAR - 1) / T_RAREP_2sra) + 1

def find_jmax(T_RAREP_4sra, T_RAREP_2sra, T_RAR, W_RAR, i):
  return np.floor((((i - 1) * T_RAREP_4sra) + T_RAR + W_RAR - 2) / T_RAREP_2sra) + 1

## **Main Function**

In [28]:
def main():
  # initail number of fallback UE
  M_FB = [0.0 for i in range(0, M - 1)]

  Ms_4SRA_list = [0.0 for i in range(0, len(M_4SRA))]
  Ps_4SRA_list = [0.0 for i in range(0, len(M_4SRA))]
  Da_4SRA_list = [0.0 for i in range(0, len(M_4SRA))]
  MiSn_4SRA_list = [0.0 for i in range(0, maxslot(T_RAREP_4sra, T_RAR_4sra, W_RAR_4sra, W_BO_4sra, retrans_4sra) + 1)]

  Ms_2SRA_list = [0.0 for i in range(0, len(M_2SRA))]
  Ps_2SRA_list = [0.0 for i in range(0, len(M_2SRA))]
  Da_2SRA_list = [0.0 for i in range(0, len(M_2SRA))]
  MiSn_2SRA_list = [0.0 for i in range(0, maxslot(T_RAREP_2sra, T_RAR_2sra, W_RAR_2sra, W_BO_2sra, retrans_2sra) + 1)]

  Ms_FB_list = [0.0 for i in range(0, len(M_2SRA))]
  Ps_FB_list = [0.0 for i in range(0, len(M_2SRA))]
  Da_FB_list = [0.0 for i in range(0, len(M_2SRA))]
  MiSn_FB_list = [0.0 for i in range(0, maxslot(T_RAREP_4sra, T_RAR_4sra, W_RAR_4sra, W_BO_4sra, retrans_4sra) + 1)]

  Ms_T_list = [0.0 for i in range(0, M - 1)]
  Ps_T_list = [0.0 for i in range(0, M - 1)]
  Da_T_list = [0.0 for i in range(0, M - 1)]
  MiSn_T_list = [0.0 for i in range(0, maxslot(T_RAREP_4sra, T_RAR_4sra, W_RAR_4sra, W_BO_4sra, retrans_4sra) + 1)]

  Slot = [i for i in range(0, maxslot(T_RAREP_4sra, T_RAR_4sra, W_RAR_4sra, W_BO_4sra, retrans_4sra) + 1)]

  for ue in range(0, M - 1):
    # initial the MiSn and MiFn: Mi[slot][num of trans + 1]
    # the last time of retrans needed to be ignored
    MiCn_4SRA = [[0.0] * int((retrans_4sra + 1) + 1) for _ in range(0, maxslot(T_RAREP_4sra, T_RAR_4sra, W_RAR_4sra, W_BO_4sra, retrans_4sra) + 1)]
    MiSn_4SRA = [[0.0] * int((retrans_4sra + 1) + 1) for _ in range(0, maxslot(T_RAREP_4sra, T_RAR_4sra, W_RAR_4sra, W_BO_4sra, retrans_4sra) + 1)]
    MiFn_4SRA = [[0.0] * int((retrans_4sra + 1) + 1) for _ in range(0, maxslot(T_RAREP_4sra, T_RAR_4sra, W_RAR_4sra, W_BO_4sra, retrans_4sra) + 1)]
    T_4SRA = [0.0 for i in range(0, maxslot(T_RAREP_4sra, T_RAR_4sra, W_RAR_4sra, W_BO_4sra, retrans_4sra) + 1)]

    MiCn_2SRA = [[0.0] * int((retrans_2sra + 1) + 1) for _ in range(0, maxslot(T_RAREP_2sra, T_RAR_2sra, W_RAR_2sra, W_BO_2sra, retrans_2sra) + 1)]
    MiSn_2SRA = [[0.0] * int((retrans_2sra + 1) + 1) for _ in range(0, maxslot(T_RAREP_2sra, T_RAR_2sra, W_RAR_2sra, W_BO_2sra, retrans_2sra) + 1)]
    MiFn_2SRA = [[0.0] * int((retrans_2sra + 1) + 1) for _ in range(0, maxslot(T_RAREP_2sra, T_RAR_2sra, W_RAR_2sra, W_BO_2sra, retrans_2sra) + 1)]
    T_2SRA = [0.0 for i in range(0, maxslot(T_RAREP_2sra, T_RAR_2sra, W_RAR_2sra, W_BO_2sra, retrans_2sra) + 1)]

    MiCn_FB = [[0.0] * int((retrans_4sra + 1) + 1) for _ in range(0, maxslot(T_RAREP_4sra, T_RAR_4sra, W_RAR_4sra, W_BO_4sra, retrans_4sra) + 1)]
    MiSn_FB = [[0.0] * int((retrans_4sra + 1) + 1) for _ in range(0, maxslot(T_RAREP_4sra, T_RAR_4sra, W_RAR_4sra, W_BO_4sra, retrans_4sra) + 1)]
    MiFn_FB = [[0.0] * int((retrans_4sra + 1) + 1) for _ in range(0, maxslot(T_RAREP_4sra, T_RAR_4sra, W_RAR_4sra, W_BO_4sra, retrans_4sra) + 1)]
    T_FB = [0.0 for i in range(0, maxslot(T_RAREP_4sra, T_RAR_4sra, W_RAR_4sra, W_BO_4sra, retrans_4sra) + 1)]

    # num of total UE in slot 0
    MiFn_2SRA[0][0] = M_2SRA[ue]
    # print(np.matrix(MiFn))

    # 2SRA procedure sure be finished before 4SRA procedure
    for i in range(0, maxslot(T_RAREP_2sra, T_RAR_2sra, W_RAR_2sra, W_BO_2sra, retrans_2sra)):
      # print("i = ", i, "\n")
      kmin = find_kmin(T_RAREP_2sra, T_RAR_2sra, W_RAR_2sra, W_BO_2sra, i)
      kmax = find_kmax(T_RAREP_2sra, T_RAR_2sra, W_RAR_2sra, W_BO_2sra, i)
      # print("kmin = ", kmin, "kmax = ", kmax, "\n")
      m_2SRA = 0
      for n in range(0, int(retrans_2sra + 1)):
        m_2SRA += MiFn_2SRA[i][n] # Mi

      if N_2SRA[0] == 0:
        E_M = 0
      else:
        E_M = np.exp(-(m_2SRA / N_2SRA[0]))

      for n in range(0, int(retrans_2sra + 1)):
        # print("n = ", n, "\n")
        k_list_2SRA = []
        k_shift_2SRA = math.ceil((T_RAR_2sra + W_RAR_2sra) / T_RAREP_2sra)
        if MiFn_2SRA[i][n] != 0:
          for k in range(int(kmin), int(kmax + 1)):
            # print("k = ", k, "\n")
            alpha = find_alpha(T_RAREP_2sra, T_RAR_2sra, W_RAR_2sra, W_BO_2sra, i, k, kmin, kmax)
            if preamble_detection == "false":
              k_list_2SRA.append(alpha * (MiFn_2SRA[i][n] * (1.0 - E_M)))
              # k_list_2SRA.append(alpha * MiCn_2SRA[i][n])
            elif preamble_detection == "true":
              P_N = 1 - (1 / math.exp(n + 1))
              total_MiFn_2SRA = 0
              for n_2sra in range(0, int(retrans_2sra + 1)):
                total_MiFn_2SRA += MiFn_2SRA[i][n_2sra] * E_M * (1 - (1 / math.exp(n_2sra + 1)))
              if total_MiFn_2SRA <= N_UL_2sra:
                k_list_2SRA.append(alpha * (MiFn_2SRA[i][n] * (1.0 - E_M * P_N)))
                # k_list_2SRA.append(alpha * MiCn_2SRA[i][n])
              else:
                k_list_2SRA.append(alpha * (MiFn_2SRA[i][n] - (((MiFn_2SRA[i][n] * E_M * P_N) / total_MiFn_2SRA) * N_UL_2sra)))
                # k_list_2SRA.append(alpha * (MiFn_2SRA[i][n] - MiSn_2SRA[i][n]))
            # print("alpha = ", alpha, "\n")
          # process the data to put it into array like type 1
          for itr_k in range(0, len(k_list_2SRA)):
            # if you find problem after adjusting the parameters, you need to set T_RAREP > T_RAR + W_RAR
            # if you still want to set T_RAREP < T_RAR + W_RAR, you need to change the value "1" in "i + itr_k + 1"
            MiFn_2SRA[i + itr_k + k_shift_2SRA][n + 1] += k_list_2SRA[(len(k_list_2SRA) - 1) - itr_k]
            # print("MiCn = ", MiFn[i][n] * (1.0 - E_M), "\n")
            # print(np.matrix(MiFn))

      for n in range(0, int(retrans_2sra + 1)):
        # move MiSn and MiCn calculation part above the kmin, kma, alpha calculation part
        if preamble_detection == "false":
          MiSn_2SRA[i][n] = MiFn_2SRA[i][n] * E_M
          MiCn_2SRA[i][n] = MiFn_2SRA[i][n] * (1.0 - E_M)
        elif preamble_detection == "true":
          P_N = 1 - (1 / math.exp(n + 1))
          # transmission at the first RA slot
          if i == 0:
            # transmit the first preamble
            if n == 0 and MiFn_2SRA[i][n] * E_M * P_N >= N_UL_2sra:
              MiSn_2SRA[i][n] = N_UL_2sra
              MiCn_2SRA[i][n] = MiFn_2SRA[i][n] - N_UL_2sra
            elif n == 0 and MiFn_2SRA[i][n] * E_M * P_N < N_UL_2sra:
              MiSn_2SRA[i][n] = MiFn_2SRA[i][n] * E_M * P_N
              MiCn_2SRA[i][n] = MiFn_2SRA[i][n] * (1.0 - E_M * P_N)
            elif n != 0:
              MiSn_2SRA[i][n] = 0
              MiCn_2SRA[i][n] = 0
          else:
            total_MiFn_2SRA = 0
            for n_2sra in range(0, int(retrans_2sra + 1)):
              total_MiFn_2SRA += MiFn_2SRA[i][n_2sra] * E_M * (1 - (1 / math.exp(n_2sra + 1)))
            # transmission at the rest of RA slot
            if total_MiFn_2SRA <= N_UL_2sra:
              MiSn_2SRA[i][n] = MiFn_2SRA[i][n] * E_M * P_N
            else:
              MiSn_2SRA[i][n] = ((MiFn_2SRA[i][n] * E_M * P_N) / total_MiFn_2SRA) * N_UL_2sra
            MiCn_2SRA[i][n] = MiFn_2SRA[i][n] - MiSn_2SRA[i][n]

      # define delay
      T_2SRA[i] = (i + 1) * T_RAREP_2sra + T_RAR_2sra + W_RAR_2sra

    # num of total UE in slot 0
    MiFn_4SRA[0][0] = M_4SRA[ue]
    # sum all the failed 2SRA UE to become total number of M_FB
    if T_RAREP_4sra < T_RAREP_2sra:
      for i in range(0, maxslot(T_RAREP_2sra, T_RAR_2sra, W_RAR_2sra, W_BO_2sra, retrans_2sra)):
        # if (MiCn_2SRA[i][3]) != 0: -> cannot use this condition, it will make error
        M_FB[ue] += MiCn_2SRA[i][retrans_2sra]
        # print(M_FB[ue], MiCn_2SRA[i])
        # jmin and jmax must minus 2 here to fix the problem caused by starting slot i = 0
        jmin = find_jmin(T_RAREP_4sra, T_RAREP_2sra, T_RAR_2sra, W_RAR_2sra, i) - 2
        jmax = find_jmax(T_RAREP_4sra, T_RAREP_2sra, T_RAR_2sra, W_RAR_2sra, i) - 2
        # print(i - 1, jmax, jmin)
        if jmax == jmin:
          MiFn_FB[i - 1][0] = MiCn_2SRA[int(jmax)][retrans_2sra]
        # print(MiFn_FB[i])
    if T_RAREP_4sra == T_RAREP_2sra:
      for i in range(0, maxslot(T_RAREP_2sra, T_RAR_2sra, W_RAR_2sra, W_BO_2sra, retrans_2sra)):
        if (MiCn_2SRA[i][retrans_2sra]) != 0:
          M_FB[ue] += MiCn_2SRA[i][retrans_2sra]
          # print(M_FB[ue], MiCn_2SRA[i])
          jmax = find_jmax(T_RAREP_4sra, T_RAREP_2sra, T_RAR_2sra, W_RAR_2sra, i) - 2
          # print(i, jmax)
          MiFn_FB[i - 1][0] = MiCn_2SRA[int(jmax)][retrans_2sra]
          # print(MiFn_FB[i])
    if T_RAREP_4sra > T_RAREP_2sra:
      # max value of i iteration could be 4sra or 2sra, they will still have the same result
      for i in range(0, maxslot(T_RAREP_4sra, T_RAR_4sra, W_RAR_4sra, W_BO_4sra, retrans_4sra)):
        if (MiCn_2SRA[i][retrans_4sra]) != 0:
          M_FB[ue] += MiCn_2SRA[i][retrans_4sra]
          # print(MiCn_2SRA[i])
          # jmin and jmax must minus 2 here to fix the problem caused by starting slot i = 0
          jmin = find_jmin(T_RAREP_4sra, T_RAREP_2sra, T_RAR_2sra, W_RAR_2sra, i) - 2
          jmax = find_jmax(T_RAREP_4sra, T_RAREP_2sra, T_RAR_2sra, W_RAR_2sra, i) - 2
          # print(i, jmin, jmax)
          for j in range(int(jmin), int(jmax + 1)):
            MiFn_FB[i - 1][0] += MiCn_2SRA[j][retrans_4sra]
          # print(MiFn_FB[i])
          # print("---------------------------------------------")
    # print(np.matrix(MiCn_2SRA))
    # print(np.matrix(MiFn_FB))

    # 4SRA procedure
    for i in range(0, maxslot(T_RAREP_4sra, T_RAR_4sra, W_RAR_4sra, W_BO_4sra, retrans_4sra)):
      # print("i = ", i, "\n")
      kmin = find_kmin(T_RAREP_4sra, T_RAR_4sra, W_RAR_4sra, W_BO_4sra, i)
      kmax = find_kmax(T_RAREP_4sra, T_RAR_4sra, W_RAR_4sra, W_BO_4sra, i)
      # print("kmin = ", kmin, "kmax = ", kmax, "\n")
      m_4SRA = 0
      m_FB = 0
      for n in range(0, int(retrans_4sra + 1)):
        m_4SRA += MiFn_4SRA[i][n] # Mi
        m_FB += MiFn_FB[i][n] # Mi

      if N_4SRA[0] == 0:
        E_M = 0
      else:
        E_M = np.exp(-((m_4SRA + m_FB) / N_4SRA[0]))

      for n in range(0, int(retrans_4sra + 1)):
        # print("n = ", n, "\n")
        k_list_4SRA = []
        k_list_FB = []
        k_shift_4SRA = math.ceil((T_RAR_4sra + W_RAR_4sra) / T_RAREP_4sra)
        k_shift_FB = math.ceil((T_RAR_4sra + W_RAR_4sra) / T_RAREP_4sra)
        if (MiFn_4SRA[i][n] + MiFn_FB[i][n]) != 0:
          for k in range(int(kmin), int(kmax + 1)):
            # print("k = ", k, "\n")
            alpha = find_alpha(T_RAREP_4sra, T_RAR_4sra, W_RAR_4sra, W_BO_4sra, i, k, kmin, kmax)
            if preamble_detection == "false":
              k_list_4SRA.append(alpha * (MiFn_4SRA[i][n] * (1.0 - E_M)))
              # k_list_4SRA.append(alpha * MiCn_4SRA[i][n])
              k_list_FB.append(alpha * (MiFn_FB[i][n] * (1.0 - E_M)))
              # k_list_FB.append(alpha * MiCn_FB[i][n])
            elif preamble_detection == "true":
              P_N = 1 - (1 / math.exp(n + 1))
              total_MiFn_4SRA_FB = 0
              for n_4sra in range(0, int(retrans_4sra + 1)):
                total_MiFn_4SRA_FB += (MiFn_4SRA[i][n_4sra] + MiFn_FB[i][n_4sra]) * E_M * (1 - (1 / math.exp(n_4sra + 1)))
              if total_MiFn_4SRA_FB <= N_UL_4sra:
                k_list_4SRA.append(alpha * (MiFn_4SRA[i][n] * (1.0 - E_M * P_N)))
                # k_list_4SRA.append(alpha * MiCn_4SRA[i][n])
                k_list_FB.append(alpha * (MiFn_FB[i][n] * (1.0 - E_M * P_N)))
                # k_list_FB.append(alpha * MiCn_FB[i][n])
              else:
                k_list_4SRA.append(alpha * (MiFn_4SRA[i][n] - (((MiFn_4SRA[i][n] * E_M * P_N) / total_MiFn_4SRA_FB) * N_UL_4sra)))
                # k_list_4SRA.append(alpha * (MiFn_4SRA[i][n] - MiSn_4SRA[i][n]))
                k_list_FB.append(alpha * (MiFn_FB[i][n] - (((MiFn_FB[i][n] * E_M * P_N) / total_MiFn_4SRA_FB) * N_UL_4sra)))
                # k_list_FB.append(alpha * (MiFn_FB[i][n] - MiSn_FB[i][n]))
            # print("alpha = ", alpha, "\n")
          # process the data to put it into array like type 1
          for itr_k in range(0, len(k_list_4SRA)):
            # itr_k must plus 1 here to fix the problem caused by starting from slot i = 0
            # if you find problem after adjusting the parameters, you need to set T_RAREP > T_RAR + W_RAR
            # if you still want to set T_RAREP < T_RAR + W_RAR, you need to change the value "1" in "i + itr_k + 1"
            MiFn_4SRA[i + itr_k + k_shift_4SRA][n + 1] += k_list_4SRA[(len(k_list_4SRA) - 1) - itr_k]
            MiFn_FB[i + itr_k + k_shift_FB][n + 1] += k_list_FB[(len(k_list_FB) - 1) - itr_k]
            # print("MiCn = ", MiFn[i][n] * (1.0 - E_M), "\n")
            # print(np.matrix(MiFn))

      for n in range(0, int(retrans_4sra + 1)):
        # move MiSn and MiCn calculation part above the kmin, kma, alpha calculation part
        if preamble_detection == "false":
          MiSn_4SRA[i][n] = MiFn_4SRA[i][n] * E_M
          MiSn_FB[i][n] = MiFn_FB[i][n] * E_M
          MiCn_4SRA[i][n] = MiFn_4SRA[i][n] * (1.0 - E_M)
          MiCn_FB[i][n] = MiFn_FB[i][n] * (1.0 - E_M)
        elif preamble_detection == "true":
          P_N = 1 - (1 / math.exp(n + 1))
          # transmission at the first RA slot
          if i == 0:
            # transmit the first preamble
            if n == 0 and MiFn_4SRA[i][n] * E_M * P_N >= N_UL_4sra:
              MiSn_4SRA[i][n] = N_UL_4sra
              MiCn_4SRA[i][n] = MiFn_4SRA[i][n] - N_UL_4sra
            elif n == 0 and MiFn_4SRA[i][n] * E_M * P_N < N_UL_4sra:
              MiSn_4SRA[i][n] = MiFn_4SRA[i][n] * E_M * P_N
              MiCn_4SRA[i][n] = MiFn_4SRA[i][n] * (1.0 - E_M * P_N)
            elif n != 0:
              MiSn_4SRA[i][n] = 0
              MiCn_4SRA[i][n] = 0
          else:
            total_MiFn_4SRA_FB = 0
            for n_4sra in range(0, int(retrans_4sra + 1)):
              total_MiFn_4SRA_FB += (MiFn_4SRA[i][n_4sra] + MiFn_FB[i][n_4sra]) * E_M * (1 - (1 / math.exp(n_4sra + 1)))
            # transmission at the rest of RA slot
            if total_MiFn_4SRA_FB <= N_UL_4sra:
              MiSn_4SRA[i][n] = MiFn_4SRA[i][n] * E_M * P_N
              MiSn_FB[i][n] = MiFn_FB[i][n] * E_M * P_N
            else:
              MiSn_4SRA[i][n] = ((MiFn_4SRA[i][n] * E_M * P_N) / total_MiFn_4SRA_FB) * N_UL_4sra
              MiSn_FB[i][n] = ((MiFn_FB[i][n] * E_M * P_N) / total_MiFn_4SRA_FB) * N_UL_4sra
            MiCn_4SRA[i][n] = MiFn_4SRA[i][n] - MiSn_4SRA[i][n]
            MiCn_FB[i][n] = MiFn_FB[i][n] - MiSn_FB[i][n]
            # print("i", i, "total_MiFn", total_MiFn_4SRA_FB)
            # print("************************************************************************************")
      # print("MiSn_4SRA", MiSn_4SRA[i])
      # print("MiFn_4SRA", MiFn_4SRA[i])
      # print("MiFn_FB", MiFn_FB[i])
      # print("-------------------------------------------------------------")
      # print("MiFn_4SRA_FB", MiFn_4SRA[i] + MiFn_FB[i])
      # print("=============================================================")

      # define delay
      T_4SRA[i] = (i + 1) * T_RAREP_4sra + T_RAR_4sra + W_RAR_4sra
      T_FB[i] = (i + 1) * T_RAREP_4sra + T_RAR_4sra + W_RAR_4sra

    # Ms, Ps, Da for 2SRA
    for i in range(0, maxslot(T_RAREP_2sra, T_RAR_2sra, W_RAR_2sra, W_BO_2sra, retrans_2sra)):
      for n in range(0, int(retrans_2sra + 1)):
        # sum the number of succeed UE for every transmission in each slot
        Ms_2SRA_list[ue] += MiSn_2SRA[i][n]
        Da_2SRA_list[ue] += MiSn_2SRA[i][n] * float(T_2SRA[i])
        MiSn_2SRA_list[i] += MiSn_2SRA[i][n]

    # Ms, Ps, Da for 4SRA
    for i in range(0, maxslot(T_RAREP_4sra, T_RAR_4sra, W_RAR_4sra, W_BO_4sra, retrans_4sra)):
      for n in range(0, int(retrans_4sra + 1)):
        # sum the number of succeed UE for every transmission in each slot
        Ms_4SRA_list[ue] += MiSn_4SRA[i][n]
        Ms_FB_list[ue] += MiSn_FB[i][n]
        Da_4SRA_list[ue] += MiSn_4SRA[i][n] * float(T_4SRA[i])
        Da_FB_list[ue] += MiSn_FB[i][n] * float(T_FB[i])
        MiSn_4SRA_list[i] += MiSn_4SRA[i][n]
        MiSn_FB_list[i] += MiSn_FB[i][n]

    # total number of succeed UE when there have k UE in the system
    Ms_4SRA_list[ue] = round(Ms_4SRA_list[ue] * 1000000) / 1000000
    Ms_2SRA_list[ue] = round(Ms_2SRA_list[ue] * 1000000) / 1000000
    Ms_FB_list[ue] = round(Ms_FB_list[ue] * 1000000) / 1000000
    Ms_T_list[ue] = round((Ms_4SRA_list[ue] + Ms_2SRA_list[ue] + Ms_FB_list[ue]) * 1000000) / 1000000

    # MiSn (only work for when 2SRA T_RAREP = 4SRA T_RAREP)
    if T_RAREP_4sra == T_RAREP_2sra:
      for i in range(0, maxslot(T_RAREP_4sra, T_RAR_4sra, W_RAR_4sra, W_BO_4sra, retrans_4sra)):
        MiSn_4SRA_list[i] = round(MiSn_4SRA_list[i] * 1000000) / 1000000
        MiSn_2SRA_list[i] = round(MiSn_2SRA_list[i] * 1000000) / 1000000
        MiSn_FB_list[i] = round(MiSn_FB_list[i] * 1000000) / 1000000
        MiSn_T_list[i] = round((MiSn_4SRA_list[i] + MiSn_2SRA_list[i] + MiSn_FB_list[i]) * 1000000) / 1000000

    if M_4SRA[ue] != 0:
      Ps_4SRA_list[ue] = round((Ms_4SRA_list[ue] / M_4SRA[ue]) * 1000000) / 1000000
    else:
      Ps_4SRA_list[ue] = 0
    if M_2SRA[ue] != 0:
      Ps_2SRA_list[ue] = round((Ms_2SRA_list[ue] / M_2SRA[ue]) * 1000000) / 1000000
    else:
      Ps_2SRA_list[ue] = 0
    if M_FB[ue] != 0:
      Ps_FB_list[ue] = round((Ms_FB_list[ue] / M_FB[ue]) * 1000000) / 1000000
    else:
      Ps_FB_list[ue] = 0
    if M_4SRA[ue] + M_2SRA[ue] != 0:
      Ps_T_list[ue] = round((Ms_4SRA_list[ue] + Ms_2SRA_list[ue] + Ms_FB_list[ue]) / (M_4SRA[ue] + M_2SRA[ue]) * 1000000) / 1000000
    else:
      Ps_T_list[ue] = 0

    if M_4SRA[ue] != 0 and Ms_4SRA_list[ue] != 0:
      Da_4SRA_list[ue] = float(Da_4SRA_list[ue]) / float(Ms_4SRA_list[ue])
    else:
      Da_4SRA_list[ue] = 0.0
    if M_2SRA[ue] != 0 and Ms_2SRA_list[ue] != 0:
      Da_2SRA_list[ue] = float(Da_2SRA_list[ue]) / float(Ms_2SRA_list[ue])
    else:
      Da_2SRA_list[ue] = 0.0
    if M_FB[ue] != 0 and Ms_FB_list[ue] != 0:
      Da_FB_list[ue] = float(Da_FB_list[ue]) / float(Ms_FB_list[ue])
    else:
      Da_FB_list[ue] = 0.0
    if M_4SRA[ue] + M_2SRA[ue] != 0:
      Da_T_list[ue] = (Da_4SRA_list[ue] * float(Ms_4SRA_list[ue]) + Da_2SRA_list[ue] * float(Ms_2SRA_list[ue]) + Da_FB_list[ue] * float(Ms_FB_list[ue])) / (float(Ms_4SRA_list[ue]) + float(Ms_2SRA_list[ue]) + float(Ms_FB_list[ue]))
    else:
      Da_T_list[ue] = 0.0

    # print(len(M), len(Ms_list), len(Ps_list), len(Da_list))

  if test_mode == 0:
    return M_4SRA, Ms_4SRA_list, Ps_4SRA_list, Da_4SRA_list, M_2SRA, Ms_2SRA_list, Ps_2SRA_list, Da_2SRA_list, M_FB, Ms_FB_list, Ps_FB_list, Da_FB_list, Ms_T_list, Ps_T_list, Da_T_list
  elif test_mode == 1:
    return Slot, MiSn_4SRA_list, MiSn_2SRA_list, MiSn_FB_list, MiSn_T_list

## **Initial Parameters**

In [37]:
test_mode = 1
preamble_detection = "false"

In [34]:
if test_mode == 0:
  # number of preamble
  N_4SRA = [44]
  N_2SRA = [10]

  # number of max retransmission
  retrans_4sra = 3 # N_PTmax - 1
  retrans_2sra = 3

elif test_mode == 1:
  # number of preamble
  N_4SRA = [54]
  N_2SRA = [54]

  # number of max retransmission
  retrans_4sra = 10 # N_PTmax - 1
  retrans_2sra = 10

# Time Duration (ms)
T_RAREP_4sra = 10
T_RAR_4sra = 2 # ra-ResponseWindow
N_RAR_4sra = 3
W_RAR_4sra = 5
N_UL_4sra = N_RAR_4sra * W_RAR_4sra
T_N_CR_4sra = 32
W_BO_4sra = 20

# Time Duration (ms)
T_RAREP_2sra = 10
T_RAR_2sra = 2 # ra-ResponseWindow
N_RAR_2sra = 3
W_RAR_2sra = 5
N_UL_2sra = N_RAR_2sra * W_RAR_2sra
T_N_CR_2sra = 32
W_BO_2sra = 20


def maxslot(T_RAREP, T_RAR, W_RAR, W_BO, retrans):
  Maxraslot = 1 + (retrans - 1) * np.ceil((T_RAR + W_RAR + W_BO) / T_RAREP)
  return int(Maxraslot) * 2 + 6
# print(maxslot(T_RAREP_4sra, T_RAR_4sra, W_RAR_4sra, W_BO_4sra, retrans_4sra))
# print(maxslot(T_RAREP_2sra, T_RAR_2sra, W_RAR_2sra, W_BO_2sra, retrans_2sra))

if test_mode == 0:
# total number of UE
  M = 1000
  # initail number of 2SRA UE
  M_4SRA = []
  M_2SRA = []
  for i in range(1, 1000):
    M_2SRA.append(i)
    M_4SRA.append(M - i)
elif test_mode == 1:
  M = 2
  M_4SRA = [500]
  M_2SRA = [500]


## **Test Result**

In [35]:
if test_mode == 0:
  columns = ["M_4SRA", "N_4SRA", "Ms_4SRA", "Ps_4SRA", "Da_4SRA", "M_2SRA", "N_2SRA", "Ms_2SRA", "Ps_2SRA", "Da_2SRA", "M_FB", "Ms_FB", "Ps_FB", "Da_FB", "Ms_T", "Ps_T", "Da_T"]
  M_4SRA, Ms_4SRA_list, Ps_4SRA_list, Da_4SRA_list, M_2SRA, Ms_2SRA_list, Ps_2SRA_list, Da_2SRA_list, M_FB, Ms_FB_list, Ps_FB_list, Da_FB_list, Ms_T_list, Ps_T_list, Da_T_list = main()

  N_4SRA_list = [N_4SRA[0] for i in range(0, M - 1)]
  N_2SRA_list = [N_2SRA[0] for i in range(0, M - 1)]

  data = {"M_4SRA" : M_4SRA, "N_4SRA" : N_4SRA_list, "Ms_4SRA" : Ms_4SRA_list, "Ps_4SRA" : Ps_4SRA_list, "Da_4SRA" : Da_4SRA_list,
          "M_2SRA" : M_2SRA, "N_2SRA" : N_2SRA_list, "Ms_2SRA" : Ms_2SRA_list, "Ps_2SRA" : Ps_2SRA_list, "Da_2SRA" : Da_2SRA_list,
          "M_FB": M_FB, "Ms_FB": Ms_FB_list, "Ps_FB": Ps_FB_list, "Da_FB": Da_FB_list,
          "Ms_T" : Ms_T_list, "Ps_T" : Ps_T_list, "Da_T" : Da_T_list}

elif test_mode == 1:
  columns = ["Slot", "M_4SRA", "N_4SRA", "MiSn_4SRA", "M_2SRA", "N_2SRA", "MiSn_2SRA", "MiSn_FB", "MiSn_T"]
  Slot, MiSn_4SRA_list, MiSn_2SRA_list, MiSn_FB_list, MiSn_T_list = main()

  M_4SRA_list = [M_4SRA[0] for i in range(0, len(Slot))]
  M_2SRA_list = [M_4SRA[0] for i in range(0, len(Slot))]

  N_4SRA_list = [N_4SRA[0] for i in range(0, len(Slot))]
  N_2SRA_list = [N_2SRA[0] for i in range(0, len(Slot))]

  data = {"Slot" : Slot,
          "M_4SRA" : M_4SRA_list, "N_4SRA" : N_4SRA_list, "MiSn_4SRA" : MiSn_4SRA_list,
          "M_2SRA" : M_2SRA_list, "N_2SRA" : N_2SRA_list, "MiSn_2SRA" : MiSn_2SRA_list,
          "MiSn_FB" : MiSn_FB_list, "MiSn_T" : MiSn_T_list}

result = pandas.DataFrame(data)
# cvs_pandas = result.to_csv("ana_testresult.csv", mode='a', index=False, header=True, columns=columns)
cvs_pandas = result.to_csv("ana_testresult.csv", mode='w+', index=False, header=True, columns=columns)

## **Plotting Figures**

In [36]:
ana = pandas.read_csv('ana_testresult.csv')
# sim = pandas.read_csv('sim_testresult.csv')
ana

,Slot,M_4SRA,N_4SRA,MiSn_4SRA,M_2SRA,N_2SRA,MiSn_2SRA,MiSn_FB,MiSn_T
0,0,500,54,0.047613,500,54,0.047613,0.0,0.095226
1,1,500,54,18.702108,500,54,18.702108,0.0,37.404216
2,2,500,54,2.157680,500,54,2.157680,0.0,4.315360
3,3,500,54,2.755693,500,54,2.755693,0.0,5.511386
4,4,500,54,6.122954,500,54,6.122954,0.0,12.245908
...,...,...,...,...,...,...,...,...,...
58,58,500,54,0.000000,500,54,0.000000,0.0,0.000000
59,59,500,54,0.000000,500,54,0.000000,0.0,0.000000
60,60,500,54,0.000000,500,54,0.000000,0.0,0.000000
61,61,500,54,0.000000,500,54,0.000000,0.0,0.000000
